In [26]:
def approx_match(time_a, time_b, slippage=0.040000000000000036):
    return abs(time_a - time_b) <= slippage

In [28]:
SAMPLE = "2442204240010034621"

In [34]:
from pathlib import Path
TEXT = Path("/Users/joregan/Playing/kbw2v")
PHONES = Path("/Users/joregan/Playing/rd_phonetic")

In [35]:
text = open(TEXT / f"{SAMPLE}_480p.json")
phones = open(PHONES / f"{SAMPLE}_480p.json")

In [36]:
import json
text_json = json.load(text)
phone_json = json.load(phones)

In [121]:
class Chunk:
    def __init__(self, chunk):
        self.text = chunk['text']
        self.start = chunk['timestamp'][0]
        self.end = chunk['timestamp'][1]
    
    def __repr__(self) -> str:
        return f"[{self.text} ({self.start}, {self.end})]"

class SimpleMerge(Chunk):
    def __init__(self, left: Chunk, right: Chunk):
        self.text = left.text
        self.phone = right.text
        self.start = left.start
        self.end = left.end
        self.diff_start = left.start - right.start
        self.diff_end = left.end - right.end

    def exact_length(self):
        return self.diff_start == 0 and self.diff_end == 0

    def __repr__(self) -> str:
        return f"[{self.text} :: {self.phone} ({self.start}, {self.end})]"

class ComplexMerge(Chunk):
    def __init__(self, left, right):
        if type(left) == list:
            self.left_chunks = left
        elif left == None:
            self.left_chunks == []
        else:
            self.left_chunks = [left]
        if type(right) == list:
            self.right_chunks = right
        elif right == None:
            self.right_chunks == []
        else:
            self.right_chunks = [right]

        self.start = self.get_start()
        self.end = self.get_end()

        self.text = " ".join([x.text for x in self.left_chunks])
        self.phone = " ".join([x.text for x in self.right_chunks])

    def get_start(self):
        if self.left_chunks == [] and self.right_chunks == []:
            return None
        if self.left_chunks == []:
            return self.right_chunks[0].start
        if self.right_chunks == []:
            return self.left_chunks[0].start
        if self.left_chunks[0].start < self.right_chunks[0].start:
            return self.left_chunks[0].start
        else:
            return self.right_chunks[0].start

    def get_end(self):
        if self.left_chunks == [] and self.right_chunks == []:
            return None
        if self.left_chunks == []:
            return self.right_chunks[-1].end
        if self.right_chunks == []:
            return self.left_chunks[-1].end
        if self.left_chunks[-1].end > self.right_chunks[-1].end:
            return self.left_chunks[-1].end
        else:
            return self.right_chunks[-1].end

    def __repr__(self) -> str:
        return f"[{self.text} :: {self.phone} ({self.start}, {self.end})]"

        

In [82]:
text_chunks = []
phone_chunks = []
for text_chunk in text_json['chunks']:
    text_chunks.append(Chunk(text_chunk))

for phone_chunk in phone_json['chunks']:
    phone_chunks.append(Chunk(phone_chunk))

In [129]:
class PhoneChunk(Chunk):
    def __init__(self, chunk):
        self.text = chunk.text
        self.start = chunk.start
        self.end = chunk.end
class WordChunk(Chunk):
    def __init__(self, chunk):
        self.text = chunk.text
        self.start = chunk.start
        self.end = chunk.end

In [150]:
merged = []

start = True

pci = 0
tci = 0

iteration = 0

while pci < len(phone_chunks) and tci < len(text_chunks):
    iteration += 1
    print("tci", tci, "pci", pci)
    print("it", iteration, "considering", text_chunks[tci], phone_chunks[pci])
    if start:
        if phone_chunks[pci].end < text_chunks[tci].start:
            merged.append(PhoneChunk(phone_chunks[pci]))
        start = False
        # pci += 1
        # tci += 1
        # continue
    else:
        am_start = approx_match(text_chunks[tci].start, phone_chunks[pci].start)
        am_end = approx_match(text_chunks[tci].end, phone_chunks[pci].end)
        if am_start and am_end:
            merged.append(SimpleMerge(text_chunks[tci], phone_chunks[pci]))
            pci += 1
            tci += 1
            continue
        elif am_start:
            print("it", iteration, tci, pci)
            cur_text = [text_chunks[tci]]
            cur_phone = [phone_chunks[pci]]
            if phone_chunks[pci].end < text_chunks[tci].end:
                while not approx_match(text_chunks[tci].end, phone_chunks[pci].end):
                    print("it", iteration, "in while")
                    pci += 1
                    if pci >= len(phone_chunks):
                        break
                    cur_phone.append(phone_chunks[pci])
                merged.append(ComplexMerge(cur_text, cur_phone))
                tci += 1
                continue
            else:
                while not approx_match(text_chunks[tci].end, phone_chunks[pci].end):
                    tci += 1
                    if tci >= len(text_chunks):
                        break
                    cur_text.append(text_chunks[tci])
                merged.append(ComplexMerge(cur_text, cur_phone))
                pci += 1
                continue
        else:
            print(text_chunks[tci], phone_chunks[pci])
    tci += 1
    pci += 1


tci 0 pci 0
it 1 considering [IX (0.02, 0.34)] [iː (0.02, 0.06)]
tci 1 pci 1
it 2 considering [DEBATT (0.66, 1.08)] [seks (0.12, 0.46)]
[DEBATT (0.66, 1.08)] [seks (0.12, 0.46)]
tci 2 pci 2
it 3 considering [MED (1.18, 1.24)] [debat (0.66, 1.12)]
[MED (1.18, 1.24)] [debat (0.66, 1.12)]
tci 3 pci 3
it 4 considering [ANLEDNING (1.38, 1.92)] [meː (1.16, 1.22)]
[ANLEDNING (1.38, 1.92)] [meː (1.16, 1.22)]
tci 4 pci 4
it 5 considering [AV (2.04, 2.18)] [anleːdnɪŋ (1.36, 1.92)]
[AV (2.04, 2.18)] [anleːdnɪŋ (1.36, 1.92)]
tci 5 pci 5
it 6 considering [INTEPELATIONSSVAR (2.38, 3.34)] [ɑːv (2.04, 2.22)]
[INTEPELATIONSSVAR (2.38, 3.34)] [ɑːv (2.04, 2.22)]
tci 6 pci 6
it 7 considering [OCH (3.84, 3.92)] [ɪntepelaɧuːnsvɑːr (2.38, 3.34)]
[OCH (3.84, 3.92)] [ɪntepelaɧuːnsvɑːr (2.38, 3.34)]
tci 7 pci 7
it 8 considering [KAMMAREN (3.96, 4.32)] [oː (3.86, 3.88)]
[KAMMAREN (3.96, 4.32)] [oː (3.86, 3.88)]
tci 8 pci 8
it 9 considering [ÖVERGÅR (4.44, 4.78)] [kamarən (3.96, 4.34)]
[ÖVERGÅR (4.44, 4.78)] [kam

In [143]:
merged

[[IX :: iː seks debat meː anleːdnɪŋ ɑːv ɪntepelaɧuːnsvɑːr oː kamarən øːvərɡoːr nʉː tɪl at debateːra stasroːdən svɑːr poː møːtæɳa s ɪntepelaɧuːnər <pa> oː viː stɑːʈa meː arbesmarknas ɔk jemstɛlheːts mɪnɪstər eːva nuːɖmark sʊsjɑːldemɔkrɑːtæɳa s <hes> svɑːr poː ɪntepelaɧuːn tvoː hɵndra niːe <sm> ɑːv roːɡər hadad lɪbərɑːləɳa arbeːtsfœ̞meːdlɪɡəns tɪl<v>jeŋlɪɡheːt ɔk viː bœ̞rja meː mɪnɪstən vɑːʂoːuːɡuːd tak frʉː tɑːlman roːɡər hadad hɑːr froːɡat mej ɔm deː fɪns noːra kɔŋkreːta moːl ɵpsata fœ̞ː tɪljeŋlɪheːtən tɪl<v> arbesfœ̞meːdlɪŋən sɔm rɪjrɪŋən fœljər ɵp viːdarə hɑː n froːɡat <pa>vɑː jɑː anseːr æːr en rɪmlɪ nɪvoː fœ̞ːr hʉː snapt maŋ kaŋ kɵna kɔma iː kɔntakt meː en arbesfmeːdlarə <pa> samt foː en oːtekɔplɪŋ <sm><pa> ɧenærelt<v> <hes> set slʉːtlɪən hɑːr an froːɡat vɪlka oːtjæːɖə rɪeːrɪŋən ɑːv seː at viːtɑː fœ̞ːr at fœ̞ betra tɪljeŋlɪheːtən fœ̞ːr ɪnskriːvna hʊs arbesfmeːdlɪŋən ɧenerelt <pa> men ɪntə mɪnst fœ̞ːr pæʂuːnər meːd fɵŋkɧʊsnesɛtnɪŋ rɪjeːrɪŋən ɑː vɑːrɪt tyːdlɪ meː at <pa> arbesmaknaspʊ